# Simulating with Census Data

# Step 1 - process your census data
On Your Own - here is an example

In [3]:
## All data files for step 2

# base_population
base_pop_data_path = ''
base_pop_mapping = ''
base_pop_out_file = 'base_population.pkl'

# household
household_data_path = ''
household_mapping = ''
household_out_file = 'household.pkl'

In [4]:
## Executing step 2

from AgentTorch.loaders import CensusDataGenerator
census_loader = CensusDataGenerator()

census_loader.generate_basepop(base_pop_data_path, base_pop_mapping, base_pop_out_file)
census_loader.generate_households(household_data_path, household_mapping, household_out_file)

population_df = census_loader.population_df # this can be used directly for simulation

In [ ]:
## Step 3 - Run an existing simulation
from AgentTorch.simulation import load_sim, run_sim
from AgentTorch.models import covid

covid_config = load_sim(covid)
run_sim(covid_config, population_df)

In [ ]:
'''Existing simulation exists - rerun in few lines of code'''
def run_sim(config, population_df):
    assert column_set
    
    config_data_path = preprocess(population_df)
    config.set_data(config_data_path)
    
    runner = Runner(config)
    runner.execute()

In [7]:
import pandas as pd
import torch


def load_population(df_path,attribute):
    """
    Load population data from a pandas dataframe
    """
    # Load population data
    df = pd.read_pickle(df_path)
    df = df['synpop'][attribute]
    df,mapping = pd.factorize(df)
    att_tensor = torch.from_numpy(df).float()
    return att_tensor,mapping

In [5]:
df_path = '/tmp/syspop_test/NYC/1/tmp/synpop.pickle'

In [ ]:
pop = load_population(df_path,'area')


In [ ]:
pop

In [ ]:
# pop = pop.values
pop = torch.from_numpy(pop).float()
pop

In [ ]:
df = pop['synpop']['area']
df = pd.get_dummies(df, columns=['area']).astype(int)
df

In [13]:
df = pd.read_pickle(df_path)
gender = df['synpop']['gender']
gender,map = pd.factorize(gender)


In [15]:
mapping_collection = {}
mapping_collection['gender'] = map
mapping_collection

dict_values([Index(['female', 'male'], dtype='object')])

In [16]:
df = pd.read_pickle(df_path)
age = df['synpop']['age']
age,map = pd.factorize(age)

In [17]:
mapping_collection['age'] = map

In [18]:
mapping_collection

{'gender': Index(['female', 'male'], dtype='object'),
 'age': Index(['U19', '20t29', '30t39', '40t49', '50t64', '65A'], dtype='object')}

In [10]:
df.keys()

dict_keys(['synpop', 'synadd'])

In [9]:
df['synpop'].keys()

Index(['area', 'age', 'gender', 'ethnicity'], dtype='object')

In [ ]:
f

In [ ]:
df['synpop']['area']

In [19]:
df = pd.read_pickle("/tmp/syspop_test/NYC/1/tmp/area.pickle")

In [22]:
att_tensor = torch.from_numpy(df.values).float()
att_tensor

tensor([ 0.,  0.,  0.,  ..., 32., 32., 32.])

# Prompt

In [ ]:
####
import os
####
import torch
import random
import numpy as np

def set_seeds(seed):    
    random.seed(seed)
    np.random.seed(seed)
    
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
set_seeds(42)
####
from accelerate import Accelerator

from accelerate.utils import gather_object
from transformers import AutoModelForCausalLM, AutoTokenizer
from statistics import mean
import torch, time, json

accelerator = Accelerator()
####
import json
from PIL import Image

import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration
from transformers import CLIPProcessor, CLIPModel


In [ ]:
prompt_template = "You are {male} of age {age} and are {ethnicity}. You live in {area}. Give your willingness to work in {industry}, denote the willingness by giving a value between 0 and 1, with 0 being not willing at all and 1 being completely willing."

In [ ]:
get_agent_age = "U19"
get_agent_gender = "Male"
get_agent_ethnicity = "White"
area = "Manhattan"
industry = "Finance"
prompt = prompt_template.format(male = get_agent_gender,age = get_agent_age,ethnicity=get_agent_ethnicity,industry=industry,area=area)

In [ ]:
prompt

In [ ]:
torch_dtype = torch.bfloat16
model_id = "llava-hf/llava-1.5-7b-hf"
processor = AutoProcessor.from_pretrained(model_id)

model = LlavaForConditionalGeneration.from_pretrained(
    model_id, 
    device_map={"": accelerator.device},
    torch_dtype=torch_dtype, 
    low_cpu_mem_usage=True,
)
    
model.eval()

In [41]:
import pandas as pd 
df = pd.read_csv("/Users/shashankkumar/Documents/GitHub/MacroEcon/simulator_data/NYC/brooklyn_unemp.csv")
df.sort_values(by=['year','month'],ascending=True,inplace=True)
arr = df['unemployment_rate'].values
tensor = torch.from_numpy(arr)
tensor = tensor.view(5,-1)
tensor[0]

,area,year,month,unemployment_rate
52,1,0,0,4.9
53,1,0,1,4.5
54,1,0,2,4.2
55,1,0,3,3.7
56,1,0,4,4.0
...,...,...,...,...
8,1,4,8,5.5
9,1,4,9,5.8
10,1,4,10,5.3
11,1,4,11,5.4


In [45]:
arr = df['unemployment_rate'].values
tensor = torch.from_numpy(arr)
tensor = tensor.view(5,-1)
tensor[0][:3]

tensor([4.9000, 4.5000, 4.2000], dtype=torch.float64)

In [44]:
tensor

tensor([[ 4.9000,  4.5000,  4.2000,  3.7000,  4.0000,  4.0000,  4.4000,  4.3000,
          3.7000,  4.0000,  3.7000,  3.4000,  4.1000],
        [ 4.2000,  4.3000,  5.4000, 12.4000, 21.7000, 17.3000, 17.7000, 15.2000,
         14.5000, 13.2000, 12.6000, 12.2000, 12.6000],
        [13.1000, 12.7000, 11.7000, 11.2000, 10.5000, 11.0000, 10.7000, 10.3000,
          8.9000,  8.4000,  7.8000,  7.6000, 10.3000],
        [ 8.0000,  7.3000,  6.5000,  5.9000,  5.6000,  5.6000,  5.7000,  5.4000,
          4.7000,  5.2000,  5.3000,  5.2000,  5.9000],
        [ 5.7000,  5.8000,  5.4000,  5.0000,  5.2000,  5.6000,  5.7000,  5.9000,
          5.5000,  5.8000,  5.3000,  5.4000,  5.5000]], dtype=torch.float64)

In [39]:
max_area = df['area'].max()
max_year = df['year'].max()
max_month = df['month'].max()
import numpy as np

data = np.zeros((max_area+1, max_year+1, max_month+1))
for _, row in df.iterrows():
    data[int(row['area'])][int(row['year'])][int(row['month'])] = row['unemployment_rate']
tensor = torch.from_numpy(data)
tensor[1][0][11]

tensor(3.4000, dtype=torch.float64)

In [7]:
df.values

array([[ 1. ,  0. ,  0. ,  4.9],
       [ 1. ,  0. ,  1. ,  4.5],
       [ 1. ,  0. ,  2. ,  4.2],
       [ 1. ,  0. ,  3. ,  3.7],
       [ 1. ,  0. ,  4. ,  4. ],
       [ 1. ,  0. ,  5. ,  4. ],
       [ 1. ,  0. ,  6. ,  4.4],
       [ 1. ,  0. ,  7. ,  4.3],
       [ 1. ,  0. ,  8. ,  3.7],
       [ 1. ,  0. ,  9. ,  4. ],
       [ 1. ,  0. , 10. ,  3.7],
       [ 1. ,  0. , 11. ,  3.4],
       [ 1. ,  0. , 12. ,  4.1],
       [ 1. ,  1. ,  0. ,  4.2],
       [ 1. ,  1. ,  1. ,  4.3],
       [ 1. ,  1. ,  2. ,  5.4],
       [ 1. ,  1. ,  3. , 12.4],
       [ 1. ,  1. ,  4. , 21.7],
       [ 1. ,  1. ,  5. , 17.3],
       [ 1. ,  1. ,  6. , 17.7],
       [ 1. ,  1. ,  7. , 15.2],
       [ 1. ,  1. ,  8. , 14.5],
       [ 1. ,  1. ,  9. , 13.2],
       [ 1. ,  1. , 10. , 12.6],
       [ 1. ,  1. , 11. , 12.2],
       [ 1. ,  1. , 12. , 12.6],
       [ 1. ,  2. ,  0. , 13.1],
       [ 1. ,  2. ,  1. , 12.7],
       [ 1. ,  2. ,  2. , 11.7],
       [ 1. ,  2. ,  3. , 11.2],
       [ 1

In [9]:
import torch

In [27]:
un = torch.from_numpy(df.values)
un = un.view(un.shape[0],1,3,2)
un[0][1]
# un = un[0].view(4,-1)
# un[3]

RuntimeError: shape '[65, 4, 3, 2, -1]' is invalid for input of size 260

In [ ]:
un.shaoe

In [ ]:
print("episode: ", episode)
num_steps_per_episode = self.config["simulation_metadata"]["num_steps_per_episode"]
self.reset()
for steps in range(num_steps_per_episode):
    self.step(1)
    unemployment_rate_list = [state['environment']['U'] for state in self.state_trajectory[-1] if state['current_substep'] == str(self.config['simulation_metadata']['num_substeps_per_step'] - 1)]
    unemployment_rate_tensor = torch.tensor(unemployment_rate_list,requires_grad=True).float()
    test_set_for_episode = self.unemployment_test_dataset[episode][:num_steps_per_episode-1].float()
    loss =  self.mse_loss(unemployment_rate_tensor, test_set_for_episode)
    loss.backward()
    self.optimizer.step()
    self.optimizer.zero_grad()